### **MULTICLASS IMAGE CLASSIFICATION - WHITE BLOOD CELLS**

----

In [152]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import pickle
import gc
import json

from collections import Counter,deque

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler

import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix

#### **GLOBAL VARIABLES**

In [153]:
DATA_PATH = 'data'
CATEGORIES = ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']
SPLITS = ['train', 'validation', 'test']
CHANNELS = ['red', 'green', 'blue']

#### **HELPER FUNCTIONS**

In [154]:
def get_raw_python_from_notebook(notebook,python=None):
    if python is None: python=notebook
    with open(notebook+'.ipynb','r') as f:
        rawpy = json.load(f)
    rawpy = [[] if c['source'] == [] else c['source'] for c in rawpy['cells'] if c['cell_type']=='code']
    for r in rawpy:
        r.extend(['\n','\n'])
    raw = [l for r in rawpy for l in r]
    with open(python+'.py', 'w') as f:
        f.write(''.join(raw))
get_raw_python_from_notebook('multiclass_classification_wbc')

In [3]:
def get_model_size(m):
    if type(m) is str:
        m = tf.keras.models.load_model(m)
    size = 0
    for layer in m.layers:
        weight_byte_heuristic = 13.69 # chosen from varied empirical evidence :)
        for w in layer.weights:
            s = w.numpy().shape
            n = 1
            for val in s:
                n *= val
            size += n
    print('Total Neurons:', f'{size:,}')
    print('Estimated Model Size:', np.round(1e-6*weight_byte_heuristic*size,2),'MB')
    #return size,int(weight_byte_heuristic*size)

In [4]:
def get_file_counts(data_path=DATA_PATH, splits=SPLITS, categories=CATEGORIES):
    dirs = {}
    for j in splits:
        dirs[j] = {}
        for i in categories:
            dirs[j][i] = os.path.join(data_path,j,i)
            print('size of', j, 'directory for', i, ':', len(os.listdir(dirs[j][i])))
        print('TOTAL length of', j, 'set :', sum([len(os.listdir(dirs[j][i])) for i in categories]))

In [5]:
def get_file_paths(data_path=DATA_PATH, splits=SPLITS, categories=CATEGORIES, df=True):
    paths = []
    for j in splits:
        for i in categories:
            d = {'category': i , 'split': j}
            p = os.path.join(data_path,j,i)
            [paths.append(dict(d,**{'file_path':os.path.join(p,f),'file':f})) for f in os.listdir(p) if 'jpeg' in f]
    if df is True: return pd.DataFrame.from_dict(paths)
    return paths

In [6]:
def extract_channels(dataset_path, path, categories=CATEGORIES, splits=SPLITS, channels=CHANNELS):
    filepaths = {}
    for s in splits:
        for c in categories:
            for f in [f for f in os.listdir(os.path.join(dataset_path,s,c)) if 'jpeg' in f]:
                p = os.path.join(dataset_path,s,c,f)
                filepaths[p] = {}
                for r in channels:
                    filepaths[p][r] = os.path.join(dataset_path,'channels',s,c,(r+f))

    img = image.load_img(path)
    dims = np.array(img).shape
    channels = np.reshape(img,(dims[0]*dims[1],3)).transpose()
    (red,green,blue) = [np.reshape(channels[i],(dims[0],dims[1])).astype(float) for i in range(3)]
    gb_mean = (0.5*(blue+green))
    rb_mean = (0.5*(red+blue))
    rg_mean = (0.5*(red+green))
    red_dominance = red-gb_mean
    green_dominance = green-rb_mean
    blue_dominance = blue-rg_mean
    red_dominance *= 255/np.max(red_dominance)
    green_dominance *= 255/np.max(green_dominance)
    blue_dominance *= 255/np.max(blue_dominance)

    for r in ['red','green','blue']:
        try:
            os.mkdir(os.path.join(dataset_path,r))
            #print('A')
        except:
            pass
        for s in splits:
            try:
                os.mkdir(os.path.join(dataset_path,r,s))
                #print('B')
            except:
                pass
            for c in categories:
                try:
                    os.mkdir(os.path.join(dataset_path,r,s,c))
                    #print('C')
                except:
                    pass

    plt.imsave(filepaths[path]['red'], red_dominance, cmap='gray')
    plt.imsave(filepaths[path]['green'], green_dominance, cmap='gray')
    plt.imsave(filepaths[path]['blue'], blue_dominance, cmap='gray')

In [7]:
def calc_performance(model, data, indices=None, names=SPLITS, original_data=None, verbose=False):
    if type(data) != list: data = [data]
    if (original_data is not None) and (type(original_data) != list): original_data = [original_data]
    if original_data is None: original_data = data
    if indices is None: indices = [s for s in range(len(data[0].classes))]
    blank_indices = [s for s in range(np.max(indices)) if s not in indices]

    classes, true_classes, accuracy_tables, accuracy, cm = {}, {}, {}, {}, {}

    for i in names:
        classes[i] = [indices[np.argmax(c)] if len(c)>1 else indices[int(np.round(c))] for c in model.predict(data[names.index(i)], verbose=False)]+blank_indices
        true_classes[i] = [indices[j] for j in data[names.index(i)].classes]+blank_indices
        accuracy_tables[i] = pd.concat([pd.DataFrame(classes[i],columns=['classes']),pd.DataFrame(true_classes[i],columns=['true_classes'])],axis=1)
        accuracy_tables[i]['accuracy'] = accuracy_tables[i].apply(lambda r: 1 if r['true_classes']==r['classes'] else 0,axis=1)
        accuracy[i] = sum(accuracy_tables[i]['accuracy'])/len(accuracy_tables[i])
        cm[i] = pd.DataFrame(confusion_matrix(true_classes[i], classes[i], normalize='true'))
        for b in blank_indices:
            cm[i].drop(b,axis=0,inplace=True)
            cm[i].drop(b,axis=1,inplace=True)

    return {
        'classes': classes,
        'true_classes': true_classes,
        'accuracy': accuracy,
        'confusion_matrix': cm
    }

In [8]:
def make_confusion_matrix(d1, d2):
    return pd.DataFrame(confusion_matrix(d1, d2, normalize='true'))

In [9]:
#### MODEL CALLBACKS: reuseable

# stop early if no improvement after 5 epochs
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    mode='max',
    restore_best_weights=True
)

# save the model with the maximum validation accuracy 
checkpoint = ModelCheckpoint(
    'models/classification_wbc1.h5',
    monitor='val_accuracy',
    verbose=1,
    mode='max', 
    save_best_only=True
)

# reduce learning rate
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy', #'val_loss',
    factor=0.1,
    patience=10,
    mode='max',
    verbose=1
)

lr_scheduler = LearningRateScheduler(lambda epoch: 1e-5 * 10**(1.5*epoch/EPOCHS))

# traverse a set of learning rate values starting from 1e-4, increasing by 10**(epoch/20) every epoch
# def lr_scheduler(epochs=100, lrs=(1e-5,1e-2)):
#     return LearningRateScheduler(
#         lambda epoch: lrs[0] * 10**(np.log10(lrs[1]/lrs[0])*epoch/epochs)
#     )

####
#### **INITIAL ATTEMPTS**

<p style="font-weight: 500; color: #556;">First, let's read in data and get it into the format we expect. The method here will augment the training data, leaving the validation and test datasets untouched.</p>

<p style="font-weight: 500; color: #556;">We'll also rescale the data into the 0-1 range:</p>

In [332]:
training_datagen = ImageDataGenerator(
    rescale=1.0/255.0, # rotation_range=40,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_val_datagen = ImageDataGenerator(rescale = 1.0/255.0)

def flow_data(generator, data_path, split,shuffle=True, classes=None, class_mode='categorical'):
    return generator.flow_from_directory(
        os.path.join(data_path,split),
        target_size=(128,128),
        classes=classes,
        class_mode=class_mode,
        batch_size=60,
        shuffle=shuffle
    )

np.random.seed(67) # use a consistent seed so shuffling gives expected results
train_data = flow_data(training_datagen, DATA_PATH, 'train')
validation_data = flow_data(test_val_datagen, DATA_PATH, 'validation')
train_data_unshuffled = flow_data(training_datagen, DATA_PATH, 'train', shuffle=False)
validation_data_unshuffled = flow_data(test_val_datagen, DATA_PATH, 'validation', shuffle=False)
test_data_unshuffled = flow_data(test_val_datagen, DATA_PATH, 'test', shuffle=False)

Found 9957 images belonging to 4 classes.
Found 1887 images belonging to 4 classes.
Found 9957 images belonging to 4 classes.
Found 1887 images belonging to 4 classes.
Found 600 images belonging to 4 classes.


In [333]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [334]:
model_1.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_65 (Conv2D)          (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d_65 (MaxPoolin  (None, 63, 63, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_66 (Conv2D)          (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_66 (MaxPoolin  (None, 30, 30, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_67 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_67 (MaxPoolin  (None, 14, 14, 128)    

In [33]:
# here, we get a proxy for the model size based on the number of neurons.  
get_model_size(model_1)

Total Neurons: 407,780
Estimated Model Size: 5.58 MB


In [36]:
model_1.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(), #'rmsprop',
    metrics = ['accuracy']
)

In [35]:
EPOCHS = 100

In [13]:
history1 = model_1.fit(
    train_data,
    epochs = EPOCHS,
    validation_data = validation_data,
    verbose = 1,
    callbacks = [reduce_lr,lr_scheduler,checkpoint]
)
# model_1.save('models/classification_wbc1.h5')

Epoch 1/100


2022-09-03 01:24:20.922164: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


166/166 [==============================] - ETA: 0s - loss: 1.3868 - accuracy: 0.2523
Epoch 1: val_accuracy improved from -inf to 0.24907, saving model to models/classification_wbc1_best.h5
166/166 [==============================] - 30s 175ms/step - loss: 1.3868 - accuracy: 0.2523 - val_loss: 1.3873 - val_accuracy: 0.2491 - lr: 5.0000e-05
Epoch 2/100
166/166 [==============================] - ETA: 0s - loss: 1.3860 - accuracy: 0.2598
Epoch 2: val_accuracy improved from 0.24907 to 0.32220, saving model to models/classification_wbc1_best.h5
166/166 [==============================] - 29s 172ms/step - loss: 1.3860 - accuracy: 0.2598 - val_loss: 1.3837 - val_accuracy: 0.3222 - lr: 5.1757e-05
Epoch 3/100
166/166 [==============================] - ETA: 0s - loss: 1.3803 - accuracy: 0.2826
Epoch 3: val_accuracy did not improve from 0.32220
166/166 [==============================] - 29s 172ms/step - loss: 1.3803 - accuracy: 0.2826 - val_loss: 1.3768 - val_accuracy: 0.2570 - lr: 5.3576e-05
Epoch 

In [52]:
# retrieve best model saved from checkpoints
model_1 = tf.keras.models.load_model('models/classification_wbc1.h5')

#####
<p style="font-weight: 500; color: #556;">Using our initial model, we can now generate predictions and plot a confusion matrix:

In [81]:
classes_1 = {}
classes_1['train'] = [np.argmax(c) for c in model1.predict(train_data_unshuffled)]
classes_1['validation'] = [np.argmax(c) for c in model1.predict(validation_data_unshuffled)]
classes_1['test'] = [np.argmax(c) for c in model1.predict(test_data_unshuffled)]

10/10 [==============================] - 1s 52ms/step


In [82]:
# here we recover the classes from the main data - it's important to make sure that they are all unshuffled
true_classes_1 = {}
true_classes_1['train'] = train_data_unshuffled.classes
true_classes_1['validation'] = validation_data_unshuffled.classes
true_classes_1['test'] = test_data_unshuffled.classes

In [83]:
accuracy_tables_1 = {}
accuracy_1 = {}
for j in SPLITS:
    accuracy_tables_1[j] = pd.concat([pd.DataFrame(classes_1[j],columns=['classes']),pd.DataFrame(true_classes_1[j],columns=['true_classes'])],axis=1)
    accuracy_tables_1[j]['accuracy'] = accuracy_tables_1[j].apply(lambda r: 1 if r['true_classes']==r['classes'] else 0,axis=1)
    accuracy_1[j] = sum(accuracy_tables_1[j]['accuracy'])/len(accuracy_tables_1[j])

In [84]:
accuracy_1

{'train': 0.9328110876770112, 'validation': 0.8696343402225755, 'test': 0.875}

#####
<p style="font-weight: 500; color: #556;">View the confusion matrices:

In [87]:
for j in SPLITS:
    print()
    print(pd.DataFrame(confusion_matrix(true_classes_1[j],classes_1[j],normalize='true')))


          0         1         2         3
0  0.778534  0.002803  0.008811  0.209852
1  0.000000  0.979863  0.014901  0.005236
2  0.000000  0.000807  0.987086  0.012107
3  0.006002  0.002401  0.005202  0.986395

          0        1         2         3
0  0.636364  0.00000  0.000000  0.363636
1  0.000000  0.97234  0.014894  0.012766
2  0.000000  0.00000  0.912766  0.087234
3  0.021097  0.00211  0.018987  0.957806

          0         1         2         3
0  0.680000  0.000000  0.000000  0.320000
1  0.000000  0.966667  0.026667  0.006667
2  0.000000  0.000000  0.873333  0.126667
3  0.006667  0.000000  0.013333  0.980000


#####
<p style="font-weight: 500; color: #556;">These outputs have been rendered using a helper function and can be accessed in this way going forward...

#####
<p style="font-weight: 500; color: #556;">Our initial results are very promising, but we can see that the model has overfit here. What can we do to resolve this? Let's look at some regularization techniques, being careful not to increase the bias too much here</p>
<p style="font-weight: 500; color: #556;">We can see that group 1 has a near perfect ability to be identified, whereas confusion exists between the remaining groups.  The spit between 0 ('EOSINOPHIL') and 3 ('NEUTROPHIL') is our largest source of confusion</p>

####
#### **SUCCESSIVE REMOVAL OF CLASSES**

<p style="font-weight: 500; color: #556;">We have seen that class LYMPHOCYTE (label 1) gets picked out very well by initial models.  This suggests that once they have been predicted, a model with only 3 classes can then be made, to reduce noise when making further predictions. This process can be repeated until we ar left with a binary model for our final 2 classes.
<p style="font-weight: 500; color: #556;">We now look at the use of a waterfall system for identifying remaining items, resulting in models with successively lower numbers of classes to predict:

In [248]:
# flow from directory using only the labels 0, 2 and 3

categories_4a = ['EOSINOPHIL', 'MONOCYTE', 'NEUTROPHIL']

train_data_4a = flow_data(training_datagen, DATA_PATH, 'train', classes=categories_4a)
validation_data_4a = flow_data(test_val_datagen, DATA_PATH, 'validation', classes=categories_4a)
train_data_unshuffled_4a = flow_data(training_datagen, DATA_PATH, 'train', shuffle=False, classes=categories_4a)
validation_data_unshuffled_4a = flow_data(test_val_datagen, DATA_PATH, 'validation', shuffle=False, classes=categories_4a)
test_data_unshuffled_4a = flow_data(test_val_datagen,DATA_PATH, 'test', shuffle=False, classes=categories_4a)

Found 7474 images belonging to 3 classes.
Found 1417 images belonging to 3 classes.
Found 7474 images belonging to 3 classes.
Found 1417 images belonging to 3 classes.
Found 450 images belonging to 3 classes.


In [250]:
# test_grid = pd.DataFrame(np.array([test_data_unshuffled.classes,
#     test_data_unshuffled.labels,
#     test_data_unshuffled.filepaths]).transpose(),
#     columns=['preds','actual','filepath'])
# test_grid

In [275]:
model_4a = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), # tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [276]:
model_4a.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(), #'rmsprop',
    metrics=['accuracy']
)

In [277]:
# save the model with the maximum validation accuracy 
checkpoint = ModelCheckpoint(
    'models/classification_wbc4a_best.h5',
    monitor='val_accuracy',
    verbose=0,
    mode='max',
    save_best_only=True
)

In [278]:
EPOCHS = 100

In [280]:
history_4a = model_4a.fit(
    train_data_4a,
    epochs=EPOCHS,
    validation_data=validation_data_4a,
    verbose=1,
    callbacks=[lr_scheduler,reduce_lr,checkpoint] # early_stopping
)

Epoch 1/100
125/125 [==============================] - 20s 159ms/step - loss: 1.0992 - accuracy: 0.3353 - val_loss: 1.0978 - val_accuracy: 0.3726 - lr: 1.0000e-05
Epoch 2/100
125/125 [==============================] - 20s 156ms/step - loss: 1.0984 - accuracy: 0.3409 - val_loss: 1.0971 - val_accuracy: 0.3846 - lr: 1.0351e-05
Epoch 3/100
125/125 [==============================] - 19s 154ms/step - loss: 1.0981 - accuracy: 0.3344 - val_loss: 1.0964 - val_accuracy: 0.4495 - lr: 1.0715e-05
Epoch 4/100
125/125 [==============================] - 19s 154ms/step - loss: 1.0974 - accuracy: 0.3578 - val_loss: 1.0955 - val_accuracy: 0.3352 - lr: 1.1092e-05
Epoch 5/100
125/125 [==============================] - 19s 152ms/step - loss: 1.0966 - accuracy: 0.3602 - val_loss: 1.0934 - val_accuracy: 0.3359 - lr: 1.1482e-05
Epoch 6/100
125/125 [==============================] - 19s 154ms/step - loss: 1.0949 - accuracy: 0.3770 - val_loss: 1.0889 - val_accuracy: 0.4820 - lr: 1.1885e-05
Epoch 7/100
125/125 [=

In [281]:
model_4a = tf.keras.models.load_model('models/classification_wbc4a_best.h5')

In [345]:
data_orig = [train_data_unshuffled, validation_data_unshuffled, test_data_unshuffled]
data_4a = [train_data_unshuffled_4a, validation_data_unshuffled_4a, test_data_unshuffled_4a]

model_results_4a = calc_performance(model_4a, data_4a, [0, 2, 3], original_data=data_orig)

In [346]:
model_results_4a['accuracy']

{'train': 0.9277591973244147,
 'validation': 0.8645980253878702,
 'test': 0.88470066518847}

In [347]:
for i in model_results_4a['confusion_matrix']:
    print(model_results_4a['confusion_matrix'][i])
    print()

          0         2         3
0  0.867040  0.004005  0.128955
2  0.001211  0.985876  0.012914
3  0.059624  0.009604  0.930772

          0         2         3
0  0.826638  0.000000  0.173362
2  0.000000  0.938298  0.061702
3  0.143460  0.027426  0.829114

          0         2         3
0  0.853333  0.000000  0.146667
2  0.006667  0.933333  0.060000
3  0.126667  0.006667  0.866667



<p style="font-weight: 500; color: #556;">Here, we have no doubt that the ability to find class 2 is more easily separable than the others.  This leaves us with just classes 0 and 3, from which we can build a binary model

In [303]:
# flow from directory using only the labels 0,3
categories_4b = ['EOSINOPHIL', 'NEUTROPHIL']

train_data_4b = flow_data(training_datagen, DATA_PATH, 'train', classes=categories_4b, class_mode='binary')
validation_data_4b = flow_data(test_val_datagen, DATA_PATH, 'validation', classes=categories_4b, class_mode='binary')
train_data_unshuffled_4b = flow_data(training_datagen, DATA_PATH, 'train', shuffle=False, classes=categories_4b, class_mode='binary')
validation_data_unshuffled_4b = flow_data(test_val_datagen, DATA_PATH, 'validation', shuffle=False, classes=categories_4b, class_mode='binary')
test_data_unshuffled_4b = flow_data(test_val_datagen, DATA_PATH, 'test', shuffle=False, classes=categories_4b, class_mode='binary')

Found 4996 images belonging to 2 classes.
Found 947 images belonging to 2 classes.
Found 4996 images belonging to 2 classes.
Found 947 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [308]:
model_4b = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), # tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [309]:
model_4b.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(), # 'rmsprop',
    metrics = ['accuracy']
)

In [310]:
# save the model with the maximum validation accuracy 
checkpoint = ModelCheckpoint(
    'models/classification_wbc4b_best.h5',
    monitor='val_accuracy',
    verbose=0,
    mode='max', 
    save_best_only=True
)

In [311]:
history_4b = model_4b.fit(
    train_data_4b,
    epochs=EPOCHS,
    validation_data=validation_data_4b,
    verbose=1,
    callbacks=[reduce_lr,lr_scheduler,checkpoint]
)

Epoch 1/100
84/84 [==============================] - 14s 160ms/step - loss: 0.6933 - accuracy: 0.4976 - val_loss: 0.6933 - val_accuracy: 0.5005 - lr: 1.0000e-05
Epoch 2/100
84/84 [==============================] - 13s 153ms/step - loss: 0.6931 - accuracy: 0.5004 - val_loss: 0.6932 - val_accuracy: 0.4952 - lr: 1.0351e-05
Epoch 3/100
84/84 [==============================] - 13s 151ms/step - loss: 0.6930 - accuracy: 0.5070 - val_loss: 0.6935 - val_accuracy: 0.5005 - lr: 1.0715e-05
Epoch 4/100
84/84 [==============================] - 13s 151ms/step - loss: 0.6931 - accuracy: 0.4974 - val_loss: 0.6932 - val_accuracy: 0.5005 - lr: 1.1092e-05
Epoch 5/100
84/84 [==============================] - 13s 152ms/step - loss: 0.6931 - accuracy: 0.5016 - val_loss: 0.6933 - val_accuracy: 0.5005 - lr: 1.1482e-05
Epoch 6/100
84/84 [==============================] - 13s 151ms/step - loss: 0.6925 - accuracy: 0.5102 - val_loss: 0.6931 - val_accuracy: 0.4995 - lr: 1.1885e-05
Epoch 7/100
84/84 [===============

In [312]:
model_4b = tf.keras.models.load_model('models/classification_wbc4b_best.h5')

In [320]:
model_results_4b = calc_performance(model_4b, [train_data_unshuffled_4b, validation_data_unshuffled_4b, test_data_unshuffled_4b], [0, 3])
model_results_4b['accuracy']

{'train': 0.9221688675470188,
 'validation': 0.8145416227608009,
 'test': 0.8145695364238411}

In [321]:
for i in model_results_4b['confusion_matrix']:
    print(model_results_4b['confusion_matrix'][i])
    print()

          0         3
0  0.875050  0.124950
3  0.030812  0.969188

          0         3
0  0.693446  0.306554
3  0.065401  0.934599

          0         3
0  0.700000  0.300000
3  0.073333  0.926667



<p style="font-weight: 500; color: #556;">We can now calculate a combined probability, allowing a model's predictions to stand for the least 'confusing' class in its matrix, and moving on to the next model otherwise, unti, the final two classes use the binary model:

In [363]:
preds_1 = [np.argmax(p) for p in model_1.predict(test_data_unshuffled, verbose=0)] # classes 0,1,2,3
preds_4a = [np.argmax(p) for p in model_4a.predict(test_data_unshuffled, verbose=0)] # classes 0,2,3 (2 is at index 1)
preds_4b = [int(np.round(p)) for p in model_4b.predict(test_data_unshuffled, verbose=0)] # classes 0,3 (3 is at index 1)

In [375]:
classes_test_combined_4 = [1 if preds_1[i] == 1 else 2 if preds_4a[i] == 1 else 3*preds_4b[i] for i in range(len(preds_1))]
combined_accuracy_4 = np.mean([1 if classes_test_combined_4[i]==test_data_unshuffled.classes[i] else 0 for i in range(len(preds_1))])

In [376]:
combined_accuracy_4

0.88

In [377]:
print(make_confusion_matrix(test_data_unshuffled.classes, classes_test_combined_4))

          0         1         2         3
0  0.700000  0.000000  0.000000  0.300000
1  0.000000  0.966667  0.026667  0.006667
2  0.000000  0.000000  0.933333  0.066667
3  0.073333  0.000000  0.006667  0.920000


#####
<p style="font-weight: 500; color: #556;">Using the 128-pixel resolution, the method of binary predictions is still not decisive enough in separating classes 0 and 3 but has shown gains elsewhere.
<p style="font-weight: 500; color: #556;">As such, we can combine these results further with our higher-resolution model (model_2), to improve our precision for class 0:

In [379]:
classes_test_combined_2_4 = [0 if classes_test_combined_2[i] == 0 else classes_test_combined_4[i] for i in range(len(preds_1))]
combined_accuracy_2_4 = np.mean([1 if classes_test_combined_2_4[i]==test_data_unshuffled.classes[i] else 0 for i in range(len(preds_1))])

In [380]:
combined_accuracy_2_4

0.905

In [381]:
print(make_confusion_matrix(test_data_unshuffled.classes, classes_test_combined_2_4))

          0         1         2         3
0  0.886667  0.000000  0.000000  0.113333
1  0.000000  0.966667  0.026667  0.006667
2  0.006667  0.000000  0.933333  0.060000
3  0.160000  0.000000  0.006667  0.833333


#####
<p style="font-weight: 500; color: #556;">Superb! We've now surpassed 90% accuracy for our test set with the right mix of models.

####
#### **FULL SET OF BINARY MODELS**

<p style="font-weight: 500; color: #556;">Here, we will attempt to treat each of our 4 classes in a 'one vs rest' fashion, building a binary model for all of them, and allowing the model with the highest score to be the winner for each test case:get_file_paths()
<p style="font-weight: 500; color: #556;">(Note: this was written using a 2-class one-hot label setup with softmax activation and a categorical crossentropy loss.  How this affects performance from actually using binary crossentropy, is yet to be tested.  This is a potential evolution for the future)

In [169]:
file_data = get_file_paths()

def flow_df(generator,file_data,shuffle=True):
    return generator.flow_from_dataframe(
        file_data,
        directory=None,
        x_col='file_path',
        y_col='classes',
        target_size=(128,128),
        classes=None,
        # class_mode='binary',
        batch_size=60,
        shuffle=shuffle
    );

binary_models = []

for c in CATEGORIES:
    file_data['classes'] = file_data['category'].apply(lambda x: '0' if x==c else '1')
    file_data_train = file_data[file_data['split']=='train'].reset_index(drop=True)
    file_data_validation = file_data[file_data['split']=='validation'].reset_index(drop=True)
    file_data_test = file_data[file_data['split']=='test'].reset_index(drop=True)

    train_data = flow_df(training_datagen,file_data_train);
    validation_data = flow_df(test_val_datagen,file_data_validation);
    train_data_unshuffled = flow_df(training_datagen,file_data_train,shuffle=False);
    validation_data_unshuffled = flow_df(test_val_datagen,file_data_validation,shuffle=False);
    test_data_unshuffled = flow_df(test_val_datagen,file_data_test,shuffle=False);

    new_model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(128,128,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    binary_models.append(new_model)

    model_path = 'models/classification_wbc_binary_'+c+'_best'

    checkpoint = ModelCheckpoint(
        model_path+'.h5',
        monitor = 'val_accuracy',
        verbose = 1,
        mode = 'max', 
        save_best_only = True
    )

    EPOCHS = 100
    binary_models[-1].compile(loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(), # 'rmsprop',
    metrics = ['accuracy'])

    model_history = binary_models[-1].fit(
        train_data,
        epochs = EPOCHS,
        validation_data = validation_data,
        verbose = 1,
        callbacks = [reduce_lr,lr_scheduler,checkpoint] # observe for the future that 'reduce_lr' and 'lr_scheduler' cannot be mixed
    )

    with open(model_path+'_history.pickle','wb') as h:
        pickle.dump(model_history.history,h,protocol=pickle.HIGHEST_PROTOCOL);

Found 9957 validated image filenames belonging to 2 classes.
Found 1887 validated image filenames belonging to 2 classes.
Found 9957 validated image filenames belonging to 2 classes.
Found 1887 validated image filenames belonging to 2 classes.
Found 600 validated image filenames belonging to 2 classes.
Epoch 1/100
166/166 [==============================] - ETA: 0s - loss: 0.5693 - accuracy: 0.7446
Epoch 1: val_accuracy improved from -inf to 0.74934, saving model to models/classification_wbc_binary_EOSINOPHIL_best.h5
166/166 [==============================] - 30s 177ms/step - loss: 0.5693 - accuracy: 0.7446 - val_loss: 0.5641 - val_accuracy: 0.7493 - lr: 5.0000e-05
Epoch 2/100
166/166 [==============================] - ETA: 0s - loss: 0.5622 - accuracy: 0.7492
Epoch 2: val_accuracy did not improve from 0.74934
166/166 [==============================] - 29s 174ms/step - loss: 0.5622 - accuracy: 0.7492 - val_loss: 0.5651 - val_accuracy: 0.7493 - lr: 5.1757e-05
Epoch 3/100
166/166 [=======

In [407]:
model_5a = tf.keras.models.load_model('models/classification_wbc_binary_EOSINOPHIL.h5')
model_5b = tf.keras.models.load_model('models/classification_wbc_binary_LYMPHOCYTE.h5')
model_5c = tf.keras.models.load_model('models/classification_wbc_binary_MONOCYTE.h5')
model_5d = tf.keras.models.load_model('models/classification_wbc_binary_NEUTROPHIL.h5')

In [408]:
preds_5a = model_5a.predict(test_data_unshuffled)[:,0][np.newaxis]
preds_5b = model_5b.predict(test_data_unshuffled)[:,0][np.newaxis]
preds_5c = model_5c.predict(test_data_unshuffled)[:,0][np.newaxis]
preds_5d = model_5d.predict(test_data_unshuffled)[:,0][np.newaxis]

10/10 [==============================] - 1s 48ms/step


In [536]:
preds_5 = np.concatenate([preds_5a, preds_5b, preds_5c, preds_5d],axis=0).T
preds_5 = [np.argmax(p) for p in preds_5]

In [537]:
accuracy_5 = np.mean([1 if preds_5[i]==test_data_unshuffled.classes[i] else 0 for i in range(len(preds_5))])
accuracy_5

0.92

In [412]:
print(make_confusion_matrix(test_data_unshuffled.classes, preds_5))

          0         1     2         3
0  0.873333  0.006667  0.00  0.120000
1  0.000000  1.000000  0.00  0.000000
2  0.000000  0.000000  0.86  0.140000
3  0.033333  0.000000  0.02  0.946667


#####
<p style="font-weight: 500; color: #556;">Outstanding - we've now achieved 92 percent!  The 'one vs rest' approach to this problem clearly bears fruit.
<p style="font-weight: 500; color: #556;">The next step is to train some additional models with a higher resolution, and tune other hyper parameters, to push out our benchmark even further. Lets try the same model architecture again, for our most challenging case, this tile using **binary_crossentropy**, and then also using a larger 240px resolution:

In [612]:
file_data = get_file_paths()

def flow_df(generator,file_data,shuffle=True):
    return generator.flow_from_dataframe(
        file_data,
        directory=None,
        x_col='file_path',
        y_col='classes',
        target_size=(240,240),
        classes=None,
        class_mode='binary',
        batch_size=60,
        shuffle=shuffle
    );

binary_models = []

for c in CATEGORIES[:1]:
    file_data['classes'] = file_data['category'].apply(lambda x: '0' if x==c else '1')
    file_data_train = file_data[file_data['split']=='train'].reset_index(drop=True)
    file_data_validation = file_data[file_data['split']=='validation'].reset_index(drop=True)
    file_data_test = file_data[file_data['split']=='test'].reset_index(drop=True)

    train_data = flow_df(training_datagen,file_data_train);
    validation_data = flow_df(test_val_datagen,file_data_validation);
    train_data_unshuffled = flow_df(training_datagen,file_data_train,shuffle=False);
    validation_data_unshuffled = flow_df(test_val_datagen,file_data_validation,shuffle=False);
    test_data_unshuffled = flow_df(test_val_datagen,file_data_test,shuffle=False);
    break
    
    new_model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(240, 240, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    binary_models.append(new_model)

    model_path = 'models/classification_wbc_binary2_'+c+'.h5'

    checkpoint = ModelCheckpoint(
        model_path,
        monitor = 'val_accuracy',
        verbose = 0,
        mode = 'max', 
        save_best_only = True
    )

    EPOCHS = 50
    lr_scheduler = LearningRateScheduler(lambda epoch: 5e-5 * 10**(2*epoch/EPOCHS))
    
    binary_models[-1].compile(loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(), # 'rmsprop',
    metrics = ['accuracy'])

    model_history = binary_models[-1].fit(
        train_data,
        epochs = EPOCHS,
        validation_data = validation_data,
        verbose = 1,
        callbacks = [lr_scheduler,checkpoint]
    )

    with open(model_path+'_history.pickle','wb') as h:
        pickle.dump(model_history.history,h,protocol=pickle.HIGHEST_PROTOCOL);

Found 9957 validated image filenames belonging to 2 classes.
Found 1887 validated image filenames belonging to 2 classes.
Found 9957 validated image filenames belonging to 2 classes.
Found 1887 validated image filenames belonging to 2 classes.
Found 600 validated image filenames belonging to 2 classes.
Epoch 1/50
166/166 [==============================] - 261s 2s/step - loss: 0.5709 - accuracy: 0.7487 - val_loss: 0.5650 - val_accuracy: 0.7493 - lr: 5.0000e-05
Epoch 2/50
166/166 [==============================] - 270s 2s/step - loss: 0.5626 - accuracy: 0.7492 - val_loss: 0.5696 - val_accuracy: 0.7493 - lr: 5.4824e-05
Epoch 3/50
166/166 [==============================] - 253s 2s/step - loss: 0.5611 - accuracy: 0.7492 - val_loss: 0.5587 - val_accuracy: 0.7493 - lr: 6.0113e-05
Epoch 4/50
166/166 [==============================] - 248s 1s/step - loss: 0.5546 - accuracy: 0.7492 - val_loss: 0.5222 - val_accuracy: 0.7493 - lr: 6.5913e-05
Epoch 5/50
166/166 [==============================] - 24

In [589]:
model_6a = tf.keras.models.load_model('models/classification_wbc_binary2_EOSINOPHIL.h5')
model_6b = tf.keras.models.load_model('models/classification_wbc_binary2_LYMPHOCYTE.h5')
model_6c = tf.keras.models.load_model('models/classification_wbc_binary2_MONOCYTE.h5')
model_6d = tf.keras.models.load_model('models/classification_wbc_binary2_NEUTROPHIL.h5')

In [626]:
preds_6a = model_6a.predict(test_data_unshuffled)
preds_6b = model_6b.predict(test_data_unshuffled)
preds_6c = model_6c.predict(test_data_unshuffled)
preds_6d = model_6d.predict(test_data_unshuffled)

10/10 [==============================] - 4s 486ms/step


In [628]:
preds_6 = np.concatenate([preds_6a, preds_6b, preds_6c, preds_6d],axis=1)
preds_6 = [np.argmin(p) for p in preds_6]

In [631]:
combined_accuracy_binary = np.mean([1 if preds_6[i]==test_data_unshuffled_2.classes[i] else 0 for i in range(len(preds_6))])
combined_accuracy_binary

0.8966666666666666

In [633]:
print(make_confusion_matrix(test_data_unshuffled_2.classes, preds_6))

          0         1         2         3
0  0.880000  0.000000  0.000000  0.120000
1  0.000000  1.000000  0.000000  0.000000
2  0.000000  0.000000  0.813333  0.186667
3  0.066667  0.026667  0.013333  0.893333


####
#### **TRANSFER LEARNING**

<p style="font-weight: 500; color: #556;">Can we get any incremental benefits from using existing models? Here we will explore using a pretrained model to see if any generic information can be learned from its convolutional layers and work to out benefit.
<p style="font-weight: 500; color: #556;">Here, we will begin to use the tf.Data.dataset format for our training, to aid speed and performance

In [161]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32
(IMG_HEIGHT,IMG_WIDTH) = (240,240)

In [162]:
train_data_7 = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH+'/train',
    labels = 'inferred',
    label_mode = 'int',
    class_names = None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT,IMG_WIDTH),
    shuffle=True,
    seed=84,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

validation_data_7 = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH+'/validation',
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT,IMG_WIDTH),
    shuffle=True,
    seed=84,
)

test_data_unshuffled_7 = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH+'/test',
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT,IMG_WIDTH),
    shuffle=False, # easier if we shuffle only when we're ready to avoid gotchas
    seed=84,
)

Found 9957 files belonging to 4 classes.
Found 1887 files belonging to 4 classes.
Found 600 files belonging to 4 classes.


In [163]:
train_data_7 = train_data_7.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE) #.batch(batch_size)
validation_data_7 = validation_data_7.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
test_data_unshuffled_7 = test_data_unshuffled_7.cache().prefetch(buffer_size = AUTOTUNE)

#####
<p style="font-weight: 600; color: #556;">FIRST BASE MODEL

In [466]:
base_model = tf.keras.applications.Xception(
    weights='imagenet',
    input_shape=(240, 240, 3),
    include_top=False)
base_model.trainable = False

inputs_new = tf.keras.Input(shape=(240, 240, 3))
x = tf.keras.applications.xception.preprocess_input(inputs_new) # gives us values in the range [-1,1]
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs_new = tf.keras.layers.Dense(4)(x) # ,activation='softmax'

model_7 = tf.keras.Model(inputs_new, outputs_new)

In [461]:
model_7.compile(
    optimizer = tf.keras.optimizers.Adam(1e-3),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics=['accuracy']
)

In [77]:
model_7.fit(
    train_data_7,
    validation_data = validation_data_7,
    batch_size = 32,
    epochs = 10
)

Epoch 1/10
312/312 [==============================] - 210s 654ms/step - loss: 1.1592 - accuracy: 0.4948 - val_loss: 1.2055 - val_accuracy: 0.4732
Epoch 2/10
312/312 [==============================] - 219s 702ms/step - loss: 0.9641 - accuracy: 0.6182 - val_loss: 1.1728 - val_accuracy: 0.4812
Epoch 3/10
312/312 [==============================] - 226s 725ms/step - loss: 0.8805 - accuracy: 0.6599 - val_loss: 1.1493 - val_accuracy: 0.4971
Epoch 4/10
312/312 [==============================] - 210s 673ms/step - loss: 0.8220 - accuracy: 0.6911 - val_loss: 1.1809 - val_accuracy: 0.4870
Epoch 5/10
312/312 [==============================] - 211s 674ms/step - loss: 0.7802 - accuracy: 0.7037 - val_loss: 1.1689 - val_accuracy: 0.4918
Epoch 6/10
312/312 [==============================] - 210s 673ms/step - loss: 0.7476 - accuracy: 0.7203 - val_loss: 1.1714 - val_accuracy: 0.5003
Epoch 7/10
312/312 [==============================] - 211s 678ms/step - loss: 0.7166 - accuracy: 0.7335 - val_loss: 1.1513 -

######
<p style="font-weight: 500; color: #556;">We can quickly see that the model, with its layers frozen, is prone to heavily overfit the training set, far more than we have observed previously.  This is hence not benefitial, and we may stand more to gain by unfreezing the model's layers and allowing all of the weights to get updated.

#####
<p style="font-weight: 600; color: #556;">UNFREEZING AND FINE-TUNING</p>

In [468]:
EPOCHS = 10

base_model.trainable = True # unfreezing *all* the layers unless there are any BatchNorms in there

model_7.compile(
    optimizer = tf.keras.optimizers.Adam(1e-5),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy']
)

early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 2,
    mode = 'min',
    restore_best_weights = True
)

lr_scheduler = LearningRateScheduler(
    lambda epoch: 5e-5 * 10**(1*epoch/EPOCHS)
)

model_7.fit(
    train_data_7,
    validation_data = validation_data_7,
    batch_size = 32,
    epochs = EPOCHS,
    callbacks = [early_stopping,lr_scheduler]
)

Epoch 1/10
312/312 [==============================] - 625s 2s/step - loss: 0.2041 - accuracy: 0.9225 - val_loss: 1.4458 - val_accuracy: 0.7875 - lr: 5.0000e-05
Epoch 2/10
312/312 [==============================] - 652s 2s/step - loss: 0.0271 - accuracy: 0.9904 - val_loss: 1.5565 - val_accuracy: 0.7716 - lr: 6.2946e-05
Epoch 3/10
312/312 [==============================] - 620s 2s/step - loss: 0.0434 - accuracy: 0.9852 - val_loss: 1.4082 - val_accuracy: 0.8776 - lr: 7.9245e-05
Epoch 4/10
312/312 [==============================] - 631s 2s/step - loss: 3.7045e-04 - accuracy: 1.0000 - val_loss: 1.8114 - val_accuracy: 0.8638 - lr: 9.9763e-05
Epoch 5/10
312/312 [==============================] - 627s 2s/step - loss: 1.9093e-05 - accuracy: 1.0000 - val_loss: 1.8800 - val_accuracy: 0.8659 - lr: 1.2559e-04


#####
<p style="font-weight: 500; color: #556;">At this point, we have achieved 100% accuracy on the training data, so there is little point in continuing further.  Our highest accuracy has given us 87.76%, which is still below the level of our previous top performance.

#####
<p style="font-weight: 600; color: #556;">USING ONLY A PART OF THE BASE MODEL</p>

#####
<p style="font-weight: 500; color: #556;">Here, we explors using just a part of the pretrained model, to see the effects of only using earlier layers before the Pooling Layers made the outputs small.  Intuitively, we my be able to gain by discarding very specific information in the later layers, and keeping more generic information
<p style="font-weight: 500; color: #556;">We'll introduce some new techniques here, including new ways to define the learning rate callbacks, as well as a method for augmentation that's compatible with the new dataset formats we're using for transfer learning

In [27]:
def learning_trajectory(epochs, start=1e-5, end=1e-3, mode='linear'):
    # modes can be linear, exponential, plateau
    if mode == 'linear':
        return LearningRateScheduler(lambda epoch: start + ((end-start)*epoch/epochs))
    elif (mode == 'exp') or (mode == 'exponential'):
        return LearningRateScheduler(lambda epoch: start * 10**(np.log10(end/start)*(epoch)/(epochs-1)))
    #elif mode == 'plateau':
    #    return LearningRateScheduler(lambda epoch: start * np.log10(epoch/epochs)/np.log10(end/start))

In [ ]:
with tf.device('CPU'):
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip(),  # 'horizontal'),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(-0.3, 0.3)
    ])

#####
<p style="font-weight: 500; color: #556;">Below we quickly inspect the layers, and can see where we shift from an image height/width of 30 down to 15

In [502]:
for l in range(25, 35):
    print(l, base_model.layers[l].output.shape, base_model.layers[l].name) #.summary() #.layers[30] #.output.shape[1]

25 (None, 30, 30, 256) add_73
26 (None, 30, 30, 256) block4_sepconv1_act
27 (None, 30, 30, 728) block4_sepconv1
28 (None, 30, 30, 728) block4_sepconv1_bn
29 (None, 30, 30, 728) block4_sepconv2_act
30 (None, 30, 30, 728) block4_sepconv2
31 (None, 30, 30, 728) block4_sepconv2_bn
32 (None, 15, 15, 728) conv2d_100
33 (None, 15, 15, 728) block4_pool
34 (None, 15, 15, 728) batch_normalization_26


#####
<p style="font-weight: 500; color: #556;">Let's try catching layer 30, right before the batch normalization layer.  We,ll use the funtional API to define a new model at the output point we desire, and then feed this model in as the base model for another new model, as follows:

In [503]:
base_model = tf.keras.applications.Xception(
    weights='imagenet',
    input_shape=(240, 240, 3),
    include_top=False
)

# here we can make the model return whichever layer we want
base_model_2 = tf.keras.Model(inputs=base_model.input, outputs=base_model.layers[30].output)
base_model_2.trainable = True

inputs_new = tf.keras.Input(shape=(240, 240, 3))
x = data_augmentation(inputs_new)  # data augmentation for tf.Data.datasets
x = tf.keras.applications.xception.preprocess_input(x)  # gives us values in the range [-1,1]
x = base_model_2(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs_new = tf.keras.layers.Dense(4)(x)  # activation='softmax'

model_8 = tf.keras.Model(inputs_new, outputs_new)

In [178]:
EPOCHS = 100

model_8.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=4,
    mode='min',
    restore_best_weights=True
)

lr_scheduler = learning_trajectory(EPOCHS, 1e-5, 1e-3, 'exp')

history = model_8.fit(
    train_data,
    validation_data=validation_data,
    batch_size=32,
    epochs=EPOCHS,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/100
312/312 [==============================] - 281s 894ms/step - loss: 1.3993 - accuracy: 0.2592 - val_loss: 1.3814 - val_accuracy: 0.2591 - lr: 9.5455e-06
Epoch 2/100
312/312 [==============================] - 279s 894ms/step - loss: 1.3767 - accuracy: 0.2991 - val_loss: 1.3723 - val_accuracy: 0.3158 - lr: 1.0000e-05
Epoch 3/100
312/312 [==============================] - 276s 884ms/step - loss: 1.3662 - accuracy: 0.3270 - val_loss: 1.3634 - val_accuracy: 0.3127 - lr: 1.0476e-05
Epoch 4/100
312/312 [==============================] - 276s 885ms/step - loss: 1.3565 - accuracy: 0.3452 - val_loss: 1.3551 - val_accuracy: 0.3763 - lr: 1.0975e-05
Epoch 5/100
312/312 [==============================] - 288s 922ms/step - loss: 1.3467 - accuracy: 0.3563 - val_loss: 1.3430 - val_accuracy: 0.3784 - lr: 1.1498e-05
Epoch 6/100
312/312 [==============================] - 302s 967ms/step - loss: 1.3346 - accuracy: 0.3709 - val_loss: 1.3340 - val_accuracy: 0.4388 - lr: 1.2045e-05
Epoch 7/100
312/

In [542]:
tf.keras.models.save_model(model_8, 'models/classification_xception_multiclass.h5')

In [45]:
preds_8 = [np.argmax(p) for p in model_8.predict(test_data_unshuffled_7, verbose=0)]
true_classes_8 = test_data_unshuffled_7.map(lambda x,y: y).unbatch().batch(600) 
true_classes_8 = iter(true_classes_8).next().numpy() # to prove they are indeed the same in the new dataset format

In [46]:
accuracy_8 = np.mean([1 if preds_8[i]==true_classes_8[i] else 0 for i in range(len(preds_8))])
accuracy_8

0.92

In [47]:
print(make_confusion_matrix(true_classes_8, preds_8))

          0    1         2         3
0  0.906667  0.0  0.000000  0.093333
1  0.000000  1.0  0.000000  0.000000
2  0.000000  0.0  0.886667  0.113333
3  0.113333  0.0  0.000000  0.886667


#####
<p style="font-weight: 500; color: #556;">We've matched the best performance from our previous model - though surpassing it has provem hard.  Class 1 has reached a point of perfect prediction on our test set, though the other 3 retain some "hard" cases to resolve.

####
#### **A SET OF BINARY MODELS USING TRANSFER LEARNING**
<p style="font-weight: 500; color: #556;">We can now combine several of our previous techniques, using transfer learning as well as leaning on a suite of binary models, to squeeze out just a little more performance if possible.
<p style="font-weight: 500; color: #556;">To resolve some compatibility issues (possibly confined to M1 Macbook GPUs), it is more performant to generate a pre-augmented dataset, rather than augment on load.  The code below tackles this issue:

#####


In [20]:
from PIL import Image

def augmenting_model(input_shape = (240, 240)):
    input_shape = input_shape+(3,)
    with tf.device('CPU'):
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip(),
            tf.keras.layers.RandomRotation(0.1),
            tf.keras.layers.RandomZoom(-0.3,0.3)
        ])
    inputs = tf.keras.Input(shape=input_shape)
    outputs = data_augmentation(inputs)
    return tf.keras.Model(inputs, outputs)

def augment_process(data, path, labels, repeat=1): # we perform augmentation as a separate exercise, to bypass GPU/CPU issues
    os.makedirs(path, exist_ok = True)
    for c in labels: os.makedirs(os.path.join(path, c), exist_ok=True)

    aug_model = augmenting_model()
    for i, batch in enumerate(data.repeat(repeat)):
        x, y = batch
        x = aug_model(x.numpy())
        for j in range(len(x)):
            l = y[j].numpy()
            if (CATEGORIES[l] in labels) or (l in labels):
                im = Image.fromarray(x[j].numpy().astype(np.uint8))
                im.save(os.path.join(path, CATEGORIES[l], ('0000'+str((i*len(x))+j))[-5:]+'.jpeg'))

    data_aug = tf.keras.utils.image_dataset_from_directory( # read back in as augmented TF dataset
        path,
        labels='inferred',
        label_mode='int',
        class_names=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=(240, 240),
        shuffle=True,
        seed=84,
        validation_split=None,
        subset=None,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False
    )

    return data_aug

In [21]:
train_data_aug = augment_process(
    train_data_7,
    os.path.join(DATA_PATH,'augmented_train'),
    CATEGORIES,
    repeat=1
)

2023-01-05 11:11:26.634655: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Found 27283 files belonging to 4 classes.


#####
<p style="font-weight: 500; color: #556;">It is also possible to mass-agment into memory before proceeding with the model development and fitting, though this can cause other performance issues and is not the approach taken.  The code is nevertheless kept here for posterity:

In [90]:
def augment_data(data, dataset_size=2500, batch_size=32, input_shape=(240, 240), write_path='augmented'):
    model_aug = augmenting_model(input_shape = input_shape)
    aug_data_x, aug_data_y = deque(), deque()
    for i, batch in enumerate(data):
        if (i > 0) and (i%50 == 0): print(i, 'augmented batches processed...')
        xt, yt = batch
        xt = np.round(model_aug(xt).numpy()*1.,0)
        yt = yt.numpy()*1.
        for j in range(len(xt)):
            aug_data_x.append(xt[j])
            aug_data_y.append(np.int32(yt[j]))
    aug_data_x, aug_data_y = np.asarray(aug_data_x), np.asarray(aug_data_y)
    train_data_augmented = []
    for i in range((len(aug_data_x)//dataset_size)+1):
        imin, imax = dataset_size*i, min(len(aug_data_x),dataset_size*(i+1))
        tf_data_x = tf.data.Dataset.from_tensor_slices(aug_data_x[imin:imax])
        tf_data_y = tf.data.Dataset.from_tensor_slices(aug_data_y[imin:imax])
        tf_data = tf.data.Dataset.zip((tf_data_x, tf_data_y)).cache().batch(batch_size).shuffle(1000).prefetch(buffer_size=AUTOTUNE)
        train_data_aug.append(tf_data)
    return train_data_aug

In [91]:
train_data_aug = augment_data(train_data)

50 augmented batches processed...
100 augmented batches processed...
150 augmented batches processed...
200 augmented batches processed...
250 augmented batches processed...
300 augmented batches processed...


#####
<p style="font-weight: 600; color: #556;">MAP MULTICLASS TO BINARY DATASETS</p>
<p style="font-weight: 500; color: #556;">We'll need to map the tensorflow dataset objects from multiclass to binary using a transform on the labels, as follows:</p>

In [650]:
def make_binary_datasets(data, pos_label):
    return data.map(lambda x,y: (x,tf.map_fn(lambda z: 1 if z==pos_label else 0, y)))

In [653]:
train_data_binary, train_data_binary_aug, val_data_binary, test_data_binary = [],[],[],[]
for j in range(len(CATEGORIES)):
    train_data_binary.append(make_binary_datasets(train_data_7, j))
    train_data_binary_aug.append(make_binary_datasets(train_data_aug, j))
    val_data_binary.append(make_binary_datasets(validation_data_7, j))
    test_data_binary.append(make_binary_datasets(test_data_unshuffled_7, j))

<p style="font-weight: 500; color: #556;">Now we'll define the new models, using the same set of trainable layrers from our previous attempts, but with a binary loss function.  We'll also experiment with a more aggressive learning rate!

In [152]:
EPOCHS = 20
lrs = (1e-4, 1e-2)

In [165]:
def make_binary_model(input_shape=(240,240), base_model_trainable=True, base_model_last_layer=None):
    input_shape = input_shape+(3,)

    base_model = tf.keras.applications.Xception(
        weights='imagenet',
        input_shape=input_shape,
        include_top=False
    )

    # here we can make the model return whichever layer we want
    if base_model_last_layer is not None:
        base_model_test = tf.keras.Model(inputs = base_model.input, outputs = base_model.get_layer(base_model_last_layer).output)
    base_model_test.trainable = base_model_trainable

    inputs_new = tf.keras.Input(shape = input_shape)
    x = tf.keras.applications.xception.preprocess_input(inputs_new)  # gives us values in the range [-1,1]
    x = base_model_test(x, training = False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(16, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs_new = tf.keras.layers.Dense(1)(x)
    model_new = tf.keras.Model(inputs_new, outputs_new)

    model_new.compile(
        optimizer = tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
        metrics = ['accuracy']
    )

    return model_new

In [166]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, mode='max', restore_best_weights=True)
lr_scheduler = learning_trajectory(EPOCHS, lrs, 'exp')

In [167]:
models_9, history_9 = [],[]

<p style="font-weight: 500; color: #556;">Having seen the limited benefits of frozen layers in our case (we may indeed be benefiting more from the model architecture than from the pretrained weights), we can just make our chosen layers trainable:

In [547]:
for i in range(4):
    models_9.append(make_binary_model(base_model_last_layer=base_model.layers[30])
    history_9.append(models_binary[i].fit(
        train_data_binary_aug[i],  # augmented training data
        validation_data=val_data_binary[i],  # non-augmented validation data :)
        batch_size=32,
        epochs=EPOCHS,
        callbacks=[lr_scheduler, early_stopping]
    ))
    tf.keras.models.save_model(models_9[i], 'models/classification_xception_binary_'+str(i)+'.h5')

Epoch 1/20
312/312 [==============================] - 591s 2s/step - loss: 0.3562 - accuracy: 0.8308 - val_loss: 0.2995 - val_accuracy: 0.9126 - lr: 1.0000e-04
Epoch 2/20
312/312 [==============================] - 587s 2s/step - loss: 0.0837 - accuracy: 0.9681 - val_loss: 0.9900 - val_accuracy: 0.9189 - lr: 1.2743e-04
Epoch 3/20
312/312 [==============================] - 587s 2s/step - loss: 0.0819 - accuracy: 0.9722 - val_loss: 0.1650 - val_accuracy: 0.9465 - lr: 1.6238e-04
Epoch 4/20
312/312 [==============================] - 585s 2s/step - loss: 0.0416 - accuracy: 0.9885 - val_loss: 0.5373 - val_accuracy: 0.7493 - lr: 2.0691e-04
Epoch 5/20
312/312 [==============================] - 585s 2s/step - loss: 0.1196 - accuracy: 0.9497 - val_loss: 0.1999 - val_accuracy: 0.9438 - lr: 2.6367e-04
Epoch 6/20
312/312 [==============================] - 585s 2s/step - loss: 0.0412 - accuracy: 0.9876 - val_loss: 0.2427 - val_accuracy: 0.9555 - lr: 3.3598e-04
Epoch 7/20
312/312 [====================

#####
<p style="font-weight: 500; color: #556;">We can also further fine-tune the models using a low learning rate:

In [659]:
FINE_EPOCHS = 20
lrs = (1e-6, 1e-6)
lr_scheduler = learning_trajectory(FINE_EPOCHS, lrs[0], lrs[1], 'exp')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=4, mode='max', restore_best_weights=True)

In [173]:
history_binary_ft = []

In [183]:
for i in range(4):
    history_binary_ft.append(models_9[i].fit(
        train_data_binary_aug[i], # augmented training data
        validation_data=val_data_binary[i], # non-augmented validation data :)
        batch_size=32,
        epochs=FINE_EPOCHS,
        initial_epoch=history_binary[i].epoch[-1]+1,
        callbacks=[lr_scheduler, early_stopping]
    ))
    tf.keras.models.save_model(models_9[i], 'models/classification_xception_binary_fine_'+str(i)+'.h5')

#####
<p style="font-weight: 500; color: #556;">All that's left is to calculate the results:

In [563]:
preds_9a = models_9[0].predict(test_data_unshuffled_7, verbose=0)[:,0][np.newaxis]
preds_9b = models_9[1].predict(test_data_unshuffled_7, verbose=0)[:,0][np.newaxis]
preds_9c = models_9[2].predict(test_data_unshuffled_7, verbose=0)[:,0][np.newaxis]
preds_9d = models_9[3].predict(test_data_unshuffled_7, verbose=0)[:,0][np.newaxis]

preds_9 = np.concatenate([preds_9a, preds_9b, preds_9c, preds_9d], axis=0).T
preds_9 = [np.argmax(p) for p in preds_9]

In [564]:
accuracy_9 = np.mean([1 if preds_9[i]==true_classes_8[i] else 0 for i in range(len(preds_9))])
accuracy_9

0.9066666666666666

In [565]:
print(make_confusion_matrix(true_classes_8, preds_9))

          0         1         2         3
0  0.933333  0.026667  0.000000  0.040000
1  0.000000  1.000000  0.000000  0.000000
2  0.000000  0.000000  0.746667  0.253333
3  0.053333  0.000000  0.000000  0.946667


#####
<p style="font-weight: 500; color: #556;">We have a diminished performance on our precision for class 2 (false-positives sneaking in, though it does have perfect recall).
<p style="font-weight: 500; color: #556;">This is a good opportunity to use our previous (non-binary) model which happened to give us a stronger precision for the class, taking its predictions and using our new model only for the other three classes:

In [571]:
preds_9_8 = [2 if preds_8[i]==2 else preds_9[i] for i in range(len(preds_9))]
accuracy_9_8 = np.mean([1 if preds_9_8[i]==true_classes_8[i] else 0 for i in range(len(preds_9))])
accuracy_9_8

0.9416666666666667

In [572]:
print(make_confusion_matrix(true_classes_8, preds_9_8))

          0         1         2         3
0  0.933333  0.026667  0.000000  0.040000
1  0.000000  1.000000  0.000000  0.000000
2  0.000000  0.000000  0.886667  0.113333
3  0.053333  0.000000  0.000000  0.946667


In [573]:
tf.math.confusion_matrix(
    true_classes_8, preds_9_8,
    num_classes=4
)

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[140,   4,   0,   6],
       [  0, 150,   0,   0],
       [  0,   0, 133,  17],
       [  8,   0,   0, 142]], dtype=int32)>

#####
<p style="font-weight: 500; color: #556;">With over 94% accuracy, the hybrid case gives us an outstandung result here!

####
#### **USING TRIPLET LOSS TO HELP RESOLVE THE MOST DIFFICULT CASES**

In [12]:
import io
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

In [14]:
def make_triplet_loss_model(input_shape=(240,240), base_model_trainable=True, base_model_last_layer=None):
    input_shape = input_shape+(3,)

    base_model = tf.keras.applications.Xception(
        weights='imagenet',
        input_shape=input_shape,
        include_top=False
    )

    # here we can make the model return whichever layer we want
    if base_model_last_layer is not None:
        base_model_test = tf.keras.Model(inputs=base_model.input, outputs=base_model.get_layer(base_model_last_layer).output)
    base_model_test.trainable = base_model_trainable

    inputs_new = tf.keras.Input(shape=input_shape)
    x = tf.keras.applications.xception.preprocess_input(inputs_new) # gives us values in the range [-1,1]
    x = base_model_test(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3)(x) # lets experiment with some stronger regularization here 
    outputs_new = tf.keras.layers.Dense(728)(x)
    model_new = tf.keras.Model(inputs_new, outputs_new)

    model_new.compile(
        optimizer = tf.keras.optimizers.Adam(),
        loss = tfa.losses.TripletSemiHardLoss()
    )

    return model_new

In [403]:
EPOCHS = 20
lrs = (1e-4, 1e-2)

In [404]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True)
lr_scheduler = learning_trajectory(EPOCHS, lrs, 'exp')

In [407]:
model_tl = make_triplet_loss_model(base_model_last_layer='block13_sepconv2_act')
model_tl.fit(
    train_data_aug, # augmented training data
    validation_data=validation_data, # non-augmented validation data :)
    batch_size=32,
    epochs=EPOCHS,
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/20
312/312 [==============================] - 609s 2s/step - loss: 0.4934 - val_loss: 0.5546 - lr: 1.0000e-04
Epoch 2/20
312/312 [==============================] - 601s 2s/step - loss: 0.1206 - val_loss: 0.5000 - lr: 1.2743e-04
Epoch 3/20
312/312 [==============================] - 604s 2s/step - loss: 0.0764 - val_loss: 0.4281 - lr: 1.6238e-04
Epoch 4/20
312/312 [==============================] - 605s 2s/step - loss: 0.0668 - val_loss: 0.4199 - lr: 2.0691e-04
Epoch 5/20
312/312 [==============================] - 592s 2s/step - loss: 0.1700 - val_loss: 0.5763 - lr: 2.6367e-04
Epoch 6/20
312/312 [==============================] - 590s 2s/step - loss: 0.0379 - val_loss: 0.7026 - lr: 3.3598e-04
Epoch 7/20
312/312 [==============================] - 634s 2s/step - loss: 0.2078 - val_loss: 0.4127 - lr: 4.2813e-04
Epoch 8/20
312/312 [==============================] - 601s 2s/step - loss: 0.2289 - val_loss: 0.4036 - lr: 5.4556e-04
Epoch 9/20
312/312 [==============================] - 59

In [151]:
tf.keras.models.save_model(model_tl, 'models/xception_tl') # TO DO - register the triplet loss function as a custom model object

In [165]:
EPOCHS = 50
lrs = (1e-5, 1e-2)

In [166]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=25, mode='max', restore_best_weights=True)
lr_scheduler = learning_trajectory(EPOCHS, lrs[0], lrs[1], 'exp')

In [ ]:
for i, train_batch in enumerate(train_data_aug):
    train_x, train_y = train_batch
    pred_tl = model_tl.predict(train_x, verbose=0)[0]
    train_y_real = train_y.numpy() if i == 0 else np.concatenate([train_y_real, train_y.numpy()], axis=0)
    pred_tl_train = pred_tl if i == 0 else np.concatenate([pred_tl_train, pred_tl], axis=0)

for i, val_batch in enumerate(validation_data_7):
    val_x, val_y = val_batch
    pred_tl = model_tl.predict(val_x, verbose = 0)[0]
    val_y_real = val_y.numpy() if i == 0 else np.concatenate([val_y_real, val_y.numpy()], axis=0)
    pred_tl_val = pred_tl if i == 0 else np.concatenate([pred_tl_val, pred_tl], axis=0)

for i, test_batch in enumerate(test_data_unshuffled_7):
    test_x, test_y = test_batch
    pred_tl = model_tl.predict(test_x, verbose = 0)[0]
    test_y_real = test_y.numpy() if i == 0 else np.concatenate([test_y_real, test_y.numpy()], axis=0)
    pred_tl_test = pred_tl if i == 0 else np.concatenate([pred_tl_test, pred_tl], axis=0)

In [ ]:
embedding_tl_train = tf.data.Dataset.from_tensor_slices((pred_tl_train, train_y_real)).batch(32)
embedding_tl_val = tf.data.Dataset.from_tensor_slices((pred_tl_val, val_y_real)).batch(32)
embedding_tl_test = tf.data.Dataset.from_tensor_slices((pred_tl_test, test_y_real)).batch(32)

In [33]:
model_tl_classify = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

# compile the model
model_tl_classify.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [34]:
history_tl = model_tl_classify.fit(
    embedding_tl_train,
    epochs=100,
    validation_data=embedding_tl_val,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/100
853/853 [==============================] - 10s 6ms/step - loss: 1.1764 - accuracy: 0.8792 - val_loss: 1.0528 - val_accuracy: 0.8765 - lr: 1.0000e-05
Epoch 2/100
853/853 [==============================] - 4s 5ms/step - loss: 0.7908 - accuracy: 0.9842 - val_loss: 0.7712 - val_accuracy: 0.8765 - lr: 1.1514e-05
Epoch 3/100
853/853 [==============================] - 4s 5ms/step - loss: 0.4626 - accuracy: 0.9850 - val_loss: 0.5800 - val_accuracy: 0.8755 - lr: 1.3257e-05
Epoch 4/100
853/853 [==============================] - 4s 5ms/step - loss: 0.2416 - accuracy: 0.9861 - val_loss: 0.5054 - val_accuracy: 0.8771 - lr: 1.5264e-05
Epoch 5/100
853/853 [==============================] - 4s 5ms/step - loss: 0.1283 - accuracy: 0.9874 - val_loss: 0.5007 - val_accuracy: 0.8771 - lr: 1.7575e-05
Epoch 6/100
853/853 [==============================] - 4s 5ms/step - loss: 0.0749 - accuracy: 0.9892 - val_loss: 0.5203 - val_accuracy: 0.8786 - lr: 2.0236e-05
Epoch 7/100
853/853 [==================

In [139]:
for i, test_batch in enumerate(embedding_tl_test):
    test_x, test_y = test_batch
    pred_tl = model_tl_classify.predict(test_x, verbose=0)
    test_y_real = test_y.numpy() if i == 0 else np.concatenate([test_y_real, test_y.numpy()], axis=0)
    pred_tl_all = pred_tl if i == 0 else np.concatenate([pred_tl_all, pred_tl], axis=0)

In [140]:
pred_tl_all = [np.argmax(p) for p in pred_tl_all]

In [141]:
print(pred_tl_all)  # delete this later

[0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 2, 2, 2, 3, 3, 3, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 

In [116]:
print(make_confusion_matrix(test_y_real, pred_tl_all))

          0    1         2         3
0  0.886667  0.0  0.000000  0.113333
1  0.000000  1.0  0.000000  0.000000
2  0.000000  0.0  0.746667  0.253333
3  0.020000  0.0  0.006667  0.973333


In [42]:
tf.math.confusion_matrix(
    test_y_real,
    pred_tl_all,
    num_classes=4
)

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[133,   0,   0,  17],
       [  0, 150,   0,   0],
       [  0,   0, 112,  38],
       [  3,   0,   1, 146]], dtype=int32)>

In [40]:
print('Accuracy:', '{:.3%}'.format(np.mean([1 if diff == 0 else 0 for diff in (pred_tl_all - test_y_real)])))

Accuracy: 90.167%


#####
<p style="font-weight: 500; color: #556;">The results of this method fall short, for now, of our best.  What happens if we supplement our weakest class (class 2) with a model which performed far better (model 8)?

In [49]:
zz = [2 if preds_8[i]==2 else pred_tl_all[i] for i in range(len(preds_8))]
print('Accuracy:', '{:.3%}'.format(np.mean([1 if diff == 0 else 0 for diff in (zz - test_y_real)])))

Accuracy: 93.667%


#####
<p style="font-weight: 500; color: #556;">We're now much closer to our previous peak performance but can we do ever better?
<p style="font-weight: 500; color: #556;">Let's try the triplet loss for just the two "hardest" classes as a binary problem, and increase the number of instances per class:

In [152]:
train_data_aug_v2 = augment_process(
    train_data_7,
    os.path.join(DATA_PATH, 'augmented_train_v2'),
    [CATEGORIES[j] for j in [2,3]],
    repeat=4
).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [53]:
validation_data = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH+'/validation',
    labels='inferred',
    label_mode='int',
    batch_size=None,
    image_size=(IMG_HEIGHT,IMG_WIDTH),
    shuffle=True,
    seed=84,
)

# subset validation data to keep only classes 2 and 3
x_tensors,y_tensors = [],[]
labels = [2,3]
for i,t in enumerate(validation_data):
    if t[1].numpy() in labels:
        x_tensors.append(t[0])
        y_tensors.append(t[1])

tf_labels = tf.constant(labels) 

validation_data_v2 = tf.data.Dataset.from_tensor_slices((x_tensors, y_tensors)).batch(32)
validation_data_v2 = validation_data_v2.map(lambda x, y: (x, tf.map_fn(lambda z: tf.cast(tf.where(tf.equal(z,tf_labels))[0][0], dtype='int32'), y)))
validation_data_v2 = validation_data_v2.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

Found 1887 files belonging to 4 classes.


In [73]:
test_data = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH+'/test',
    labels='inferred',
    label_mode='int',
    batch_size=None,
    image_size=(IMG_HEIGHT,IMG_WIDTH),
    shuffle=False,
    seed=84,
)

# subset validation data to keep only classes 2 and 3
x_tensors,y_tensors = [],[]
labels = [2,3]
for i,t in enumerate(test_data):
    if t[1].numpy() in labels:
        x_tensors.append(t[0])
        y_tensors.append(t[1])
    
@tf.function
def map_labels(labels,z):
    tf_labels = tf.constant(labels)
    return tf.cast(tf.where(tf.equal(z,tf_labels))[0][0],dtype='int32')

test_data = tf.data.Dataset.from_tensor_slices((x_tensors, y_tensors)).batch(32)
test_data = test_data.map(lambda x, y: (x, tf.map_fn(lambda z: map_labels(labels,z), y)))
test_data = test_data.cache().prefetch(buffer_size = AUTOTUNE)

Found 600 files belonging to 4 classes.


In [121]:
EPOCHS = 20
lrs = (1e-7, 1e-5)  # using a VERY low learning rate here to address observed sensitivity of the loss function
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
lr_scheduler = learning_trajectory(EPOCHS, lrs[0], lrs[1], 'exp')

In [122]:
model_tl_binary = make_triplet_loss_model(base_model_last_layer='block13_sepconv2_act')
model_tl_binary.fit(
    train_data_aug_v2, # augmented training data
    validation_data=validation_data_v2, # non-augmented validation data :)
    batch_size=32,
    epochs=EPOCHS,
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/20
622/622 [==============================] - 1190s 2s/step - loss: 0.9266 - val_loss: 0.9631 - lr: 1.0000e-07
Epoch 2/20
622/622 [==============================] - 1173s 2s/step - loss: 0.9211 - val_loss: 0.9631 - lr: 1.2743e-07
Epoch 3/20
622/622 [==============================] - 1133s 2s/step - loss: 0.9118 - val_loss: 0.9634 - lr: 1.6238e-07
Epoch 4/20
622/622 [==============================] - 1140s 2s/step - loss: 0.8955 - val_loss: 0.9630 - lr: 2.0691e-07
Epoch 5/20
622/622 [==============================] - 1103s 2s/step - loss: 0.8790 - val_loss: 0.9602 - lr: 2.6367e-07
Epoch 6/20
622/622 [==============================] - 1102s 2s/step - loss: 0.8565 - val_loss: 0.9509 - lr: 3.3598e-07
Epoch 7/20
622/622 [==============================] - 1102s 2s/step - loss: 0.8102 - val_loss: 0.9214 - lr: 4.2813e-07
Epoch 8/20
622/622 [==============================] - 1100s 2s/step - loss: 0.7213 - val_loss: 0.6854 - lr: 5.4556e-07
Epoch 9/20
622/622 [============================

In [81]:
tf.keras.models.save_model(model_tl_binary, 'models/xception_tl_binary')

In [55]:
model_tl_binary = tf.keras.models.load_model('models/xception_tl_binary')

In [74]:
for i, train_batch in enumerate(train_data_aug_v2):
    train_x, train_y = train_batch
    pred_tl = model_tl_binary.predict(train_x, verbose = 0)
    train_y_real_binary = train_y.numpy() if i == 0 else np.concatenate([train_y_real_binary, train_y.numpy()], axis = 0)
    pred_tl_train_binary = pred_tl if i == 0 else np.concatenate([pred_tl_train_binary, pred_tl], axis = 0)

for i, val_batch in enumerate(validation_data_v2):
    val_x, val_y = val_batch
    pred_tl = model_tl_binary.predict(val_x, verbose = 0)
    val_y_real_binary = val_y.numpy() if i == 0 else np.concatenate([val_y_real_binary, val_y.numpy()], axis = 0)
    pred_tl_val_binary = pred_tl if i == 0 else np.concatenate([pred_tl_val_binary, pred_tl], axis = 0)

for i, test_batch in enumerate(test_data):  # we actually test on all 4 classes again
    test_x, test_y = test_batch
    pred_tl = model_tl_binary.predict(test_x, verbose = 0)
    test_y_real_binary = test_y.numpy() if i == 0 else np.concatenate([test_y_real_binary, test_y.numpy()], axis = 0)
    pred_tl_test_binary = pred_tl if i == 0 else np.concatenate([pred_tl_test_binary, pred_tl], axis = 0)

In [89]:
embedding_tl_binary_train = tf.data.Dataset.from_tensor_slices((pred_tl_train_binary, train_y_real_binary)).batch(32)
embedding_tl_binary_val = tf.data.Dataset.from_tensor_slices((pred_tl_val_binary, val_y_real_binary)).batch(32)
embedding_tl_binary_test = tf.data.Dataset.from_tensor_slices((pred_tl_test_binary, test_y_real_binary)).batch(32)

In [95]:
model_tl_binary_classify = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile the model
model_tl_binary_classify.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = ['accuracy']
)

In [96]:
EPOCHS = 50
lrs = (1e-6, 1e-4)

early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 10, mode = 'max', restore_best_weights = True)
lr_scheduler = learning_trajectory(EPOCHS, lrs[0], lrs[1], 'exp')

In [97]:
history_tl = model_tl_binary_classify.fit(
    embedding_tl_binary_train,
    epochs = EPOCHS,
    validation_data = embedding_tl_binary_val,
    callbacks = [early_stopping, lr_scheduler]
)

Epoch 1/50
1091/1091 [==============================] - 7s 6ms/step - loss: 0.5167 - accuracy: 0.8146 - val_loss: 0.4818 - val_accuracy: 0.9195 - lr: 1.0000e-06
Epoch 2/50
1091/1091 [==============================] - 6s 5ms/step - loss: 0.3807 - accuracy: 0.9550 - val_loss: 0.3803 - val_accuracy: 0.9470 - lr: 1.0985e-06
Epoch 3/50
1091/1091 [==============================] - 5s 5ms/step - loss: 0.2818 - accuracy: 0.9773 - val_loss: 0.3062 - val_accuracy: 0.9534 - lr: 1.2068e-06
Epoch 4/50
1091/1091 [==============================] - 5s 5ms/step - loss: 0.2107 - accuracy: 0.9838 - val_loss: 0.2566 - val_accuracy: 0.9492 - lr: 1.3257e-06
Epoch 5/50
1091/1091 [==============================] - 5s 5ms/step - loss: 0.1602 - accuracy: 0.9854 - val_loss: 0.2244 - val_accuracy: 0.9386 - lr: 1.4563e-06
Epoch 6/50
1091/1091 [==============================] - 6s 5ms/step - loss: 0.1240 - accuracy: 0.9864 - val_loss: 0.2044 - val_accuracy: 0.9301 - lr: 1.5999e-06
Epoch 7/50
1091/1091 [============

In [142]:
tf.keras.models.save_model(model_tl_binary_classify, 'models/xception_tl_binary_classify')

INFO:tensorflow:Assets written to: models/xception_tl_binary_classify/assets


INFO:tensorflow:Assets written to: models/xception_tl_binary_classify/assets


In [117]:
for i, test_batch in enumerate(embedding_tl_binary_test):
    test_x, test_y = test_batch
    pred_tl = model_tl_binary_classify.predict(test_x, verbose = 0)
    test_y_real_binary = test_y.numpy() if i == 0 else np.concatenate([test_y_real_binary, test_y.numpy()], axis = 0)
    pred_tl_all_binary = pred_tl if i == 0 else np.concatenate([pred_tl_all_binary, pred_tl], axis = 0)

In [143]:
pred_tl_all_binary = 2+np.asarray([int(p) for p in np.round(pred_tl_all_binary.flatten(),0)])
pred_tl_all_binary_final = np.concatenate([pred_tl_all[:300],pred_tl_all_binary],axis=0)
print('Accuracy:','{:.3%}'.format(np.mean([1 if diff == 0 else 0 for diff in (pred_tl_all_binary_final - test_y_real)])))

Accuracy: 94.500%


In [146]:
print(pred_tl_all_binary_final)  # can remove this later

[0 3 0 3 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 3 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 3 0
 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0
 3 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 3
 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 3 3 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3
 2 2 2 2 2 2 2 2 3 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2
 2 3 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 3 2 2 2 2 2 2 3 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2
 3 2 2 2 2 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 

In [144]:
print(make_confusion_matrix(test_y_real, pred_tl_all_binary_final))

          0    1         2         3
0  0.886667  0.0  0.000000  0.113333
1  0.000000  1.0  0.000000  0.000000
2  0.000000  0.0  0.913333  0.086667
3  0.000000  0.0  0.020000  0.980000


In [121]:
tf.math.confusion_matrix(
    test_y_real_binary+2,
    pred_tl_all_binary,
    num_classes=4
)

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[  0,   0,   0,   0],
       [  0,   0,   0,   0],
       [  0,   0, 137,  13],
       [  0,   0,   3, 147]], dtype=int32)>

#####
<p style="font-weight: 500; color: #556;">We've achieved our best score using this technique, 94.5%.  There are potentially more gains to be had in making binary cases for all classes - this, however, I will leave for later.

####
#### **CONCLUSIONS**